# Xem kiểu dữ liệu

In [15]:
import json
# Xem kiểu dữ liệu
def display_json_structure(data, indent=0):
    """Recursively display the structure of a JSON object."""
    if isinstance(data, dict):
        for key, value in data.items():
            print("  " * indent + f"{key}: {type(value).__name__}")
            display_json_structure(value, indent + 1)
    elif isinstance(data, list):
        print("  " * indent + f"[List of {len(data)} {type(data[0]).__name__ if data else 'Empty'}]")
        if data:
            display_json_structure(data[0], indent + 1)

# Main function

# Load input and info data
with open('./instances_attributes_train2020.json', 'r') as file:
    data = json.load(file)

# Display structure of JSON files
print("Input JSON structure:")
display_json_structure(data)

Input JSON structure:
annotations: list
  [List of 333401 dict]
    image_id: int
    category_id: int
    attribute_ids: list
      [List of 1 int]
    segmentation: list
      [List of 1 list]
        [List of 44 int]
    bbox: list
      [List of 4 float]
    area: int
    iscrowd: int
    id: int
images: list
  [List of 45623 dict]
    id: int
    width: int
    height: int
    file_name: str
    license: int
    time_captured: str
    original_url: str
    isstatic: int
    kaggle_id: str
info: dict
  year: int
  version: str
  description: str
  contributor: str
  url: str
  date_created: str
licenses: list
  [List of 12 dict]
    url: str
    id: int
    name: str
categories: list
  [List of 46 dict]
    id: int
    name: str
    supercategory: str
    level: int
    taxonomy_id: str
attributes: list
  [List of 294 dict]
    id: int
    name: str
    supercategory: str
    level: int
    taxonomy_id: str


# Thông kê từng nhãn

In [2]:
import json
from collections import Counter

# Đọc file JSON
with open('./instances_attributes_train2020.json', 'r') as file:
    data = json.load(file)

# Đếm số lượng theo từng category_id trong annotations
category_counts = Counter(annotation['category_id'] for annotation in data['annotations'])

# Thêm thông tin số lượng vào từng category trong categories
for category in data['categories']:
    category_id = category['id']
    count = category_counts.get(category_id, 0)
    category['count'] = count

# In ra từng category với số lượng > 5000
for category in data['categories']:
    if category['count'] > 1000:
        print(f"{category['id']} {category['name']}: {category['count']}")


0 shirt, blouse: 6161
1 top, t-shirt, sweatshirt: 16548
2 sweater: 1494
3 cardigan: 1107
4 jacket: 7833
6 pants: 12414
7 shorts: 2756
8 skirt: 5046
9 coat: 3124
10 dress: 18739
13 glasses: 4855
14 hat: 2518
15 headband, head covering, hair accessory: 3470
16 tie: 1457
17 glove: 1385
18 watch: 3389
19 belt: 6851
21 tights, stockings: 4326
22 sock: 2582
23 shoe: 46374
24 bag, wallet: 7217
25 scarf: 1374
27 hood: 1226
28 collar: 10159
29 lapel: 5972
31 sleeve: 59448
32 pocket: 27179
33 neckline: 34258
34 buckle: 3300
35 zipper: 7991
36 applique: 3529
37 bead: 5084
39 flower: 1367
42 rivet: 4893
43 ruffle: 2407


# Lọc một số nhãn cần thiết

In [2]:
import json

# Đọc file JSON
with open('./instances_attributes_train2020.json', 'r') as file:
    data = json.load(file)

# Danh sách các category_id cần giữ lại
categories_to_keep = {
    0: "shirt, blouse",
    1: "top, t-shirt, sweatshirt",
    4: "jacket",
    6: "pants",
    8: "skirt",
    10: "dress",
    23: "shoe",
    24: "bag, wallet"
}

# Lọc annotations theo category_id
filtered_annotations = [
    {
        "image_id": annotation["image_id"],
        "category_id": annotation["category_id"],
        "bbox": annotation["bbox"],
        "id": annotation["id"]
    }
    for annotation in data['annotations']
    if annotation['category_id'] in categories_to_keep
]

# Lấy danh sách image_id từ filtered_annotations
image_ids_to_keep = {annotation["image_id"] for annotation in filtered_annotations}

# Lọc images theo image_id
filtered_images = [
    image
    for image in data['images']
    if image['id'] in image_ids_to_keep
]

# Lọc categories theo categories_to_keep
filtered_categories = [
    {
        "id": category["id"],
        "name": category["name"]
    }
    for category in data['categories']
    if category["id"] in categories_to_keep
]

# Tạo file JSON mới với cấu trúc yêu cầu
filtered_data = {
    "annotations": filtered_annotations,
    "images": filtered_images,
    "categories": filtered_categories
}

# Ghi file JSON mới
with open('./8-labels-cloth-classification/annotations.json', 'w') as output_file:
    json.dump(filtered_data, output_file, indent=4)

print("Trích xuất dữ liệu thành công!")


Trích xuất dữ liệu thành công!


# Chuyển mã id và chuyển định dạng dữ liệu sang dạng

Chuyển sang dạng
[{
        "imgPath": "d845a7b11a3.jpg",
        "width": 1024,
        "height": 685,
        "labels": [0, 1, 1, 1, 0, 0, 1, 0],
        "bbox": [
            {
                "label": 6,
                "labelText": "shoe"
                "bbox": [minW, minH, maxW, maxH]
            }
        ]
}]

In [3]:
import json

# Đọc file JSON đã được lọc
with open('./8-labels-cloth-classification/annotations.json', 'r') as file:
    data = json.load(file)

# Mapping category_id sang chỉ mục mới và tên
mapping = {
    0: (0, "shirt, blouse"),
    1: (1, "top, t-shirt, sweatshirt"),
    4: (2, "jacket"),
    6: (3, "pants"),
    8: (4, "skirt"),
    10: (5, "dress"),
    23: (6, "shoe"),
    24: (7, "bag, wallet")
}

# Tạo dict để tra cứu category_id -> (label, labelText)
category_mapping = {k: v for k, v in mapping.items()}

# Tạo dữ liệu mới
formatted_data = []

# Duyệt qua danh sách images
for image in data['images']:
    image_id = image['id']
    # Lọc annotations liên quan đến ảnh này
    relevant_annotations = [
        annotation for annotation in data['annotations'] if annotation['image_id'] == image_id
    ]
    
    # Tạo danh sách labels và bbox
    labels = [0] * len(mapping)  # Mặc định labels chứa toàn 0
    bbox_list = []
    
    for annotation in relevant_annotations:
        category_id = annotation['category_id']
        mapped_label, label_text = category_mapping[category_id]
        
        # Cập nhật labels
        labels[mapped_label] = 1
        
        # Chuyển bbox từ [x, y, w, h]
        x_min, y_min, w, h = annotation['bbox']
        bbox_list.append({
            "label": mapped_label,
            "labelText": label_text,
            "bbox": [x_min, y_min, w, h]
        })
    
    # Thêm thông tin ảnh vào danh sách mới
    formatted_data.append({
        "imgPath": image['file_name'],
        "width": image['width'],
        "height": image['height'],
        "labels": labels,
        "bbox": bbox_list
    })

# Ghi file JSON mới
with open('./8-labels-cloth-classification/custom.json', 'w') as output_file:
    json.dump(formatted_data, output_file, indent=4)

print("Chuyển đổi dữ liệu thành công!")


Chuyển đổi dữ liệu thành công!


# Chia tập dữ liệu

In [5]:
# Modified script to handle cases where images are not found in either directory
import os
import json
import shutil
import random

# Load the annotations file
annotations_file = './8-labels-cloth-classification/annotations2.json'
try:
    with open(annotations_file, 'r') as file:
        annotations = json.load(file)
except FileNotFoundError:
    raise FileNotFoundError(f"File {annotations_file} not found. Please check the path and try again.")

# Define directories
source_dirs = ["data/images", "train"]
destination_dir = './8-labels-cloth-classification/imgs'
os.makedirs(destination_dir, exist_ok=True)

# Filter, copy images, and drop annotations with missing images
filtered_annotations = []
for annotation in annotations:
    img_path = annotation['imgPath']
    found = False
    for source_dir in source_dirs:
        full_path = os.path.join(source_dir, img_path)
        if os.path.exists(full_path):
            dest_path = os.path.join(destination_dir, img_path)
            os.makedirs(os.path.dirname(dest_path), exist_ok=True)
            shutil.copy(full_path, dest_path)
            filtered_annotations.append(annotation)
            found = True
            break
    if not found:
        print(f"Image {img_path} not found in both directories. Skipping this annotation.")

# Split data into train, val, and test sets
random.shuffle(filtered_annotations)
total = len(filtered_annotations)
train_split = int(total * 0.8)
val_split = int(total * 0.1)

train_data = filtered_annotations[:train_split]
val_data = filtered_annotations[train_split:train_split + val_split]
test_data = filtered_annotations[train_split + val_split:]

# Save the splits into respective files
output_files = {
    'train': './8-labels-cloth-classification/train/data.json',
    'val': './8-labels-cloth-classification/val/data.json',
    'test': './8-labels-cloth-classification/test/data.json'
}

for split, output_file in output_files.items():
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    with open(output_file, 'w') as file:
        json.dump(eval(f"{split}_data"), file, indent=4)

len(train_data), len(val_data), len(test_data)


(35290, 4411, 4412)

# Thông kê số lượng từng nhãn trên từng tập dữ liệu

In [5]:
import json
from collections import Counter

def count_labels_and_images(file_path, file_path2=None):
    # Đọc dữ liệu từ file JSON đầu tiên
    with open(file_path, 'r') as file:
        data = json.load(file)

    label_counts = Counter()
    total_images = len(data)

    # Đếm số lượng từng nhãn trong file đầu tiên
    for item in data:
        for i, label in enumerate(item['labels']):
            if label == 1:  # Nếu nhãn tồn tại
                label_counts[i] += 1

    # Nếu có file thứ hai, tiếp tục đếm
    if file_path2 is not None:
        with open(file_path2, 'r') as file:
            data2 = json.load(file)
        total_images += len(data2)
        for item in data2:
            for i, label in enumerate(item['labels']):
                if label == 1:  # Nếu nhãn tồn tại
                    label_counts[i] += 1

    return total_images, label_counts

# Hàm tính tổng số lượng nhãn
def calculate_total_labels(label_counts):
    return sum(label_counts.values())

# Hàm in kết quả
def print_counts(name, images, labels):
    total_labels = calculate_total_labels(labels)
    print(f"=== {name} ===")
    print(f"Tổng số ảnh: {images}")
    print(f"Tổng số nhãn: {total_labels}")
    print("Số lượng từng nhãn (đã sắp xếp):")
    for label, count in sorted(labels.items(), key=lambda x: x[0]):
        print(f"  Label {label}: {count}")
    print()

# Đường dẫn đến các file JSON
train_file = './8-labels-cloth-classification/train/data.json'
val_file = './8-labels-cloth-classification/val/data.json'
test_file = './8-labels-cloth-classification/test/data.json'

# Đếm số lượng ảnh và nhãn
train_images, train_labels = count_labels_and_images(train_file)
val_images, val_labels = count_labels_and_images(val_file)
test_images, test_labels = count_labels_and_images(test_file)

# Tổng hợp dữ liệu từ train và val
testval_images, testval_labels = count_labels_and_images(test_file, val_file)

# In kết quả
print_counts("Train", train_images, train_labels)
print_counts("Validation", val_images, val_labels)
print_counts("Test", test_images, test_labels)
print_counts("Test + Validation", testval_images, testval_labels)


=== Train ===
Tổng số ảnh: 35290
Tổng số nhãn: 77466
Số lượng từng nhãn (đã sắp xếp):
  Label 0: 4881
  Label 1: 12913
  Label 2: 6150
  Label 3: 9879
  Label 4: 4039
  Label 5: 14968
  Label 6: 19128
  Label 7: 5508

=== Validation ===
Tổng số ảnh: 4411
Tổng số nhãn: 9713
Số lượng từng nhãn (đã sắp xếp):
  Label 0: 620
  Label 1: 1624
  Label 2: 804
  Label 3: 1219
  Label 4: 483
  Label 5: 1844
  Label 6: 2426
  Label 7: 693

=== Test ===
Tổng số ảnh: 4412
Tổng số nhãn: 9762
Số lượng từng nhãn (đã sắp xếp):
  Label 0: 614
  Label 1: 1639
  Label 2: 788
  Label 3: 1249
  Label 4: 512
  Label 5: 1854
  Label 6: 2400
  Label 7: 706

=== Test + Validation ===
Tổng số ảnh: 8823
Tổng số nhãn: 19475
Số lượng từng nhãn (đã sắp xếp):
  Label 0: 1234
  Label 1: 3263
  Label 2: 1592
  Label 3: 2468
  Label 4: 995
  Label 5: 3698
  Label 6: 4826
  Label 7: 1399



# Tạo thư mục Image

In [10]:
import shutil
import json
import os
from collections import Counter, defaultdict

# Đường dẫn tới file JSON gốc và thư mục gốc
json_path = "/8-labels-cloth-classification/annotations.json"
images_dir_path = "train"

# Danh sách các categories
categories = [
    "shirt, blouse",
    "top, t-shirt, sweatshirt",
    "jacket",
    "pants",
    "skirt",
    "dress",
    "shoe",
    "bag, wallet"
]

# Thư mục để lưu các file JSON tách ra và ảnh
output_dir_ano = "./8-labels-cloth-classification/yolo/annotations"
output_dir_img = "./8-labels-cloth-classification/yolo/imgs"
os.makedirs(output_dir_ano, exist_ok=True)
os.makedirs(output_dir_img, exist_ok=True)

# Đọc file JSON gốc
with open(json_path, "r") as file:
    data = json.load(file)

# Đếm tần suất xuất hiện của mỗi `image_id`
image_id_counter = Counter(anno["image_id"] for anno in data["annotations"])

# Gom nhóm `annotations` theo `image_id`
image_id_to_annotations = defaultdict(list)
for anno in data["annotations"]:
    image_id_to_annotations[anno["image_id"]].append(anno)

# Tách annotations theo category và đảm bảo đủ 1000 phần tử
for category in categories:
    # Lọc annotations theo category
    category_annotations = [anno for anno in data["annotations"] if anno["cate"] == category]
    
    # Sắp xếp theo tần suất của `image_id`, giảm dần
    category_annotations.sort(key=lambda x: image_id_counter[x["image_id"]], reverse=True)
    
    # Chỉ chọn 1 annotation duy nhất cho mỗi `image_id` để tránh trùng lặp
    unique_annotations = {}
    for anno in category_annotations:
        if len(unique_annotations) >= 1000:
            break
        if anno["image_id"] not in unique_annotations:
            unique_annotations[anno["image_id"]] = anno

    # Nếu chưa đủ 1000 phần tử, tiếp tục thêm từ các `image_id` chưa đủ
    while len(unique_annotations) < 1000:
        for image_id, annos in image_id_to_annotations.items():
            if len(unique_annotations) >= 1000:
                break
            if image_id not in unique_annotations:
                unique_annotations[image_id] = annos[0]

    # Lấy danh sách annotations cuối cùng
    filtered_annotations = list(unique_annotations.values())
    
    # Copy ảnh và kiểm tra nếu ảnh tồn tại
    for anno in filtered_annotations:
        file_name = anno["file_name"]
        src_path = os.path.join(images_dir_path, file_name)
        
        if os.path.exists(src_path):  # Chỉ thêm nếu ảnh tồn tại
            dest_path = os.path.join(output_dir_img, file_name)
            if not os.path.exists(dest_path):  # Tránh copy lại file đã tồn tại
                shutil.copy2(src_path, dest_path)
                print(f"Đã copy: {file_name} sang {output_dir_img}")
        else:
            print(f"Ảnh không tồn tại, bỏ qua: {file_name}")
    
    count = len(filtered_annotations)  # Đếm số lượng phần tử cuối cùng
    if count > 0:
        # Ghi file JSON cho category hiện tại
        output_data = {"annotations": filtered_annotations}
        output_file = os.path.join(output_dir_ano, f"{category.replace(', ', '_').replace(' ', '_')}.json")
        with open(output_file, "w") as outfile:
            json.dump(output_data, outfile, indent=4)
        print(f"Đã tạo file JSON: {output_file} với {count} phần tử.")
    else:
        print(f"Không có dữ liệu hợp lệ cho category: {category}")


Ảnh không tồn tại, bỏ qua: 99f88be671d654e27e6f144f2009dfef.jpg
Đã copy: 7d3199e21e96eb01cd92dfd96fb28841.jpg sang ./dataset_1000/imgs
Đã copy: ce58e988f108dd47f6c137cf76378812.jpg sang ./dataset_1000/imgs
Đã copy: 5378e5aa5d74a32e1f2824b719ceb7c8.jpg sang ./dataset_1000/imgs
Đã copy: d6a1033585538bcae99eab1c892c4845.jpg sang ./dataset_1000/imgs
Đã copy: 85e269e0a9e5b22123e57913bb27b2df.jpg sang ./dataset_1000/imgs
Đã copy: 80e4eb1383efa26100280b1c465c20a3.jpg sang ./dataset_1000/imgs
Đã copy: ba5f951394e6f52e9aef5329d756dba5.jpg sang ./dataset_1000/imgs
Đã copy: 954c60dbdb9f497f6a0c9e8f5de47c9c.jpg sang ./dataset_1000/imgs
Đã copy: 9dbf8b888b3a5eca34999887c1c97989.jpg sang ./dataset_1000/imgs
Ảnh không tồn tại, bỏ qua: b62d7df7c5dc23d93efe909960dcde52.jpg
Ảnh không tồn tại, bỏ qua: fcfa6554b1ab39dedb278e166c4742fb.jpg
Ảnh không tồn tại, bỏ qua: 1e948adc21b8b29c3a2d0fc43c81726e.jpg
Ảnh không tồn tại, bỏ qua: 4d77cb97f6e366b8fd7c685e78bd25d0.jpg
Ảnh không tồn tại, bỏ qua: a0b711e99e6580

# Tạo dữ liệu cho yolo

In [9]:
import os
import shutil
import json

# Đường dẫn đầu vào
train_input_path = "./8-labels-cloth-classification/train/data.json"
val_input_path = "./8-labels-cloth-classification/val/data.json"
test_input_path = "./8-labels-cloth-classification/test/data.json"
imgs_dir = "./8-labels-cloth-classification/imgs"

# Đường dẫn đầu ra
output_dirs = {
    "train": {"images": "./8-labels-cloth-classification/yolo/train/images/", "labels": "./8-labels-cloth-classification/yolo/train/labels/"},
    "val": {"images": "./8-labels-cloth-classification/yolo/val/images/", "labels": "./8-labels-cloth-classification/yolo/val/labels/"},
    "test": {"images": "./8-labels-cloth-classification/yolo/test/images/", "labels": "./8-labels-cloth-classification/yolo/test/labels/"},
}

# Hàm chuẩn hóa tọa độ bbox
def normalize_bbox(bbox, img_width, img_height):
    x_min, y_min, box_width, box_height = bbox
    x_center = (x_min + box_width / 2) / img_width
    y_center = (y_min + box_height / 2) / img_height
    width = box_width / img_width
    height = box_height / img_height
    return x_center, y_center, width, height

# Hàm xử lý dữ liệu
# Hàm xử lý dữ liệu với việc thêm các bộ đếm
def process_data(input_path, output_images_dir, output_labels_dir):
    with open(input_path, "r") as f:
        data = json.load(f)
    
    os.makedirs(output_images_dir, exist_ok=True)
    os.makedirs(output_labels_dir, exist_ok=True)

    num_images_copied = 0
    num_txt_files_created = 0

    for item in data:
        img_path = item["imgPath"]
        width, height = item["width"], item["height"]
        labels = item["bbox"]

        # Sao chép ảnh
        src_img_path = os.path.join(imgs_dir, img_path)
        dst_img_path = os.path.join(output_images_dir, img_path)
        shutil.copy(src_img_path, dst_img_path)
        num_images_copied += 1

        # Tạo file label
        label_file_path = os.path.join(output_labels_dir, os.path.splitext(img_path)[0] + ".txt")
        with open(label_file_path, "w") as label_file:
            for label in labels:
                class_id = label["label"]
                bbox = label["bbox"]
                normalized_bbox = normalize_bbox(bbox, width, height)
                label_file.write(f"{class_id} {' '.join(map(str, normalized_bbox))}\n")
            num_txt_files_created += 1

    print(f"Tệp input '{input_path}' có {len(data)} phần tử.")
    print(f"Đã sao chép {num_images_copied} ảnh vào '{output_images_dir}'.")
    print(f"Đã tạo {num_txt_files_created} tệp TXT vào '{output_labels_dir}'.")

# Xử lý từng tập dữ liệu
process_data(train_input_path, output_dirs["train"]["images"], output_dirs["train"]["labels"])
process_data(val_input_path, output_dirs["val"]["images"], output_dirs["val"]["labels"])
process_data(test_input_path, output_dirs["test"]["images"], output_dirs["test"]["labels"])

print("Chuyển đổi dữ liệu hoàn tất!")



Tệp input './8-labels-cloth-classification/train/data.json' có 35290 phần tử.
Đã sao chép 35290 ảnh vào './8-labels-cloth-classification/yolo/train/images/'.
Đã tạo 35290 tệp TXT vào './8-labels-cloth-classification/yolo/train/labels/'.
Tệp input './8-labels-cloth-classification/val/data.json' có 4411 phần tử.
Đã sao chép 4411 ảnh vào './8-labels-cloth-classification/yolo/val/images/'.
Đã tạo 4411 tệp TXT vào './8-labels-cloth-classification/yolo/val/labels/'.
Tệp input './8-labels-cloth-classification/test/data.json' có 4412 phần tử.
Đã sao chép 4412 ảnh vào './8-labels-cloth-classification/yolo/test/images/'.
Đã tạo 4412 tệp TXT vào './8-labels-cloth-classification/yolo/test/labels/'.
Chuyển đổi dữ liệu hoàn tất!
